In [2]:
#Utilities main function
import utility
import os
original_wd = os.getcwd()

### **Main functions**

### **Sentimental Analysis Pipeline**

##### Bert Model

In [2]:
from sentimental_analysis.bert.train import *

[nltk_data] Downloading package punkt to /Users/gjj980/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/gjj980/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/gjj980/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gjj980/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##### Non-Bert Model

In [3]:
path = os.path.join(original_wd, 'sentimental_analysis\\non_bert')
os.chdir(path)
%run train
os.chdir(original_wd)

[10, 0.9044021911014027]
Training dataset has been loaded successfully

---------------------------------
LogisticRegression model succesfully trained

---------------------------------

model_accuracy:   0.63
model_precision:   0.88
model_auc:   0.68
model_f1score:   0.70
sensitivity:   0.58
specificity:   0.78

---------------------------------

Threshold parameter tuning

Prediction using best threshold for accuracy
-------------------------

model_accuracy:   0.75
model_precision:   0.76
model_auc:   0.56
model_f1score:   0.85
sensitivity:   0.96
specificity:   0.16
Best threshold for accuracy: 0.14051379451626203
Accuracy score at best threshold: 0.7502295684113865

---------------------------------



### **Topic Modelling Pipeline**

##### Bert Model

##### Non-Bert Model

### **Dockerised API**

In [ ]:
###Create a scorring function with

def score(csv_file):
    #Call Sentimental_analysis via Docker API

    #Return csv file name as reviews_test_predictions_DSAPES.csv
    #Text, Time, Sentimental Probabilities, Sentiments
    pass